Team Members:


Ch Gnaneshwar
cs24mtech11015

Saswata Mishra
cs24mtech12001

##ASSIGNMENT_4


Implement algorithm to find minimum weight perfect matching in a edge weighted bipartite graph, you need to implement the method discussed in class.

Input: CSV file with 3 rows and number of columns is number of edges.
            Each column corresponds to a edge.
            First and second rows are vertex ids of the edges, and third row is the edge weights. Note that vertex ids are integers.
              
Output:  You need to print values of dual variables at each iteration.

Instructions: 1. You need to implement in google Collab and share the link
                       2. Max two members in a group. You need to mention roll number of group members at the top of the code

In [ ]:
import numpy as np
import pandas as pd


Given edge lists:

U: left node ids

V: right node ids

W: weights

We construct a cost matrix and run the primal–dual Hungarian method

In [ ]:
def load_edge_list(filepath):
    """Load an edge-list CSV and return U, V, W arrays.

    Expected CSV format (no header):
    - Row 0: U (left node ids, integers)
    - Row 1: V (right node ids, integers)
    - Row 2: W (edge weights, floats)

    Returns:
        U, V, W as numpy arrays (U and V as int, W as float).
    """
    raw = pd.read_csv(filepath, header=None)
    U = raw.iloc[0].astype(int).to_numpy()
    V = raw.iloc[1].astype(int).to_numpy()
    W = raw.iloc[2].astype(float).to_numpy()
    return U, V, W


I have converted the edge-list into a square cost matrix, padding with a large penalty for missing edges.
This ensures the Hungarian algorithm can compute a perfect matching

In [ ]:
def create_cost_matrix(U, V, W):
    """Construct a square cost matrix from edge lists.

    The function maps unique left and right labels to row/column indices
    and fills a square matrix of dimension `dim = max(n_left, n_right)`.
    Missing edges are filled with a large penalty `big_M` so the
    Hungarian algorithm can compute a perfect matching.

    Args:
        U, V, W: arrays of equal length describing edges (u, v, weight).

    Returns:
        C: (dim x dim) numpy array with costs
        left_labels: sorted unique left node ids
        right_labels: sorted unique right node ids
    """
    left_labels = sorted(set(U))
    right_labels = sorted(set(V))

    nL, nR = len(left_labels), len(right_labels)
    dim = max(nL, nR)

    left_index = {lab: i for i, lab in enumerate(left_labels)}
    right_index = {lab: i for i, lab in enumerate(right_labels)}

    big_M = max(W) * 10**6 + 5
    C = np.full((dim, dim), big_M, dtype=float)

    for uu, vv, ww in zip(U, V, W):
        C[left_index[uu], right_index[vv]] = ww

    return C, left_labels, right_labels


This is the main algorithm.
It performs:

-Dual initialization

-Alternating-tree BFS

-Slack updates

-Dual updates

-Augmentation

It finally returns:

-partner_L (left → right match)

-partner_R (right → left match)

-final dual values (u, v)

In [ ]:
def hungarian_primal_dual(cost):
    """Compute minimum weight perfect matching using the primal-dual Hungarian method.

    Implements the algorithm presented in class via constructing an alternating
    tree (BFS), maintaining slacks, performing dual updates, and augmenting
    matchings until every left vertex is matched.

    Args:
        cost: (n x n) numpy array of costs for matching left->right.

    Returns:
        partner_L: length-n array mapping left indices to matched right index
        partner_R: length-n array mapping right indices to matched left index
        u, v: final dual variable arrays for left and right sides
    """
    n = cost.shape[0]
    u = np.min(cost, axis=1).copy()
    v = np.zeros(n, dtype=float)

    partner_L = -np.ones(n, dtype=int)
    partner_R = -np.ones(n, dtype=int)

    for root in range(n):
        if partner_L[root] != -1:
            continue

        seen_L = np.zeros(n, dtype=bool)
        seen_R = np.zeros(n, dtype=bool)
        pred = -np.ones(n, dtype=int)

        seen_L[root] = True
        queue = [root]

        slack = np.full(n, np.inf)
        slack_from = np.full(n, -1, dtype=int)

        for j in range(n):
            slack[j] = cost[root, j] - u[root] - v[j]
            slack_from[j] = root

        while True:
            while queue:
                i = queue.pop(0)
                for j in range(n):
                    if seen_R[j]:
                        continue
                    red = cost[i, j] - u[i] - v[j]

                    if red < slack[j]:
                        slack[j] = red
                        slack_from[j] = i

                    if abs(red) < 1e-12:
                        seen_R[j] = True
                        pred[j] = i

                        if partner_R[j] == -1:
                            cur = j
                            while cur != -1:
                                i_prev = pred[cur]
                                nxt = partner_L[i_prev]
                                partner_L[i_prev] = cur
                                partner_R[cur] = i_prev
                                cur = nxt
                            break
                        else:
                            nxt_left = partner_R[j]
                            if not seen_L[nxt_left]:
                                seen_L[nxt_left] = True
                                queue.append(nxt_left)
                else:
                    continue
                break

            if partner_L[root] != -1:
                break

            unvisited_R = np.where(~seen_R)[0]
            delta = np.min(slack[unvisited_R])

            u[seen_L] += delta
            v[seen_R] -= delta
            slack[~seen_R] -= delta

            for j in range(n):
                if not seen_R[j] and abs(slack[j]) < 1e-12:
                    seen_R[j] = True
                    pred[j] = slack_from[j]

                    if partner_R[j] == -1:
                        cur = j
                        while cur != -1:
                            i_prev = pred[cur]
                            nxt = partner_L[i_prev]
                            partner_L[i_prev] = cur
                            partner_R[cur] = i_prev
                            cur = nxt
                        break
                    else:
                        nxt_left = partner_R[j]
                        if not seen_L[nxt_left]:
                            seen_L[nxt_left] = True
                            queue.append(nxt_left)

    return partner_L, partner_R, u, v


In [ ]:
filepath = "/content/Testcase.csv"

U, V, W = load_edge_list(filepath)
C, left_ids, right_ids = create_cost_matrix(U, V, W)

matchL, matchR, u_final, v_final = hungarian_primal_dual(C)

print("Cost matrix:\n", C)
print("Matching (left → right):", matchL)

total_cost = sum(C[i, matchL[i]] for i in range(C.shape[0]))
print("\nTotal Minimum Cost:", total_cost)


Cost matrix:
 [[8. 4. 7. 9.]
 [5. 2. 3. 6.]
 [9. 7. 8. 5.]
 [7. 5. 6. 4.]]
Matching (left → right): [1 2 3 0]

Total Minimum Cost: 19.0


In [ ]:
print("Final dual values u:", u_final)
print("Final dual values v:", v_final)

print("\nFinal Matching (left_id → right_id):")
for i, j in enumerate(matchL):
    if i < len(left_ids) and j < len(right_ids):
        print(f"{left_ids[i]}  →  {right_ids[j]}")


Final dual values u: [6. 4. 8. 7.]
Final dual values v: [ 0. -2. -1. -3.]

Final Matching (left_id → right_id):
10  →  2
20  →  3
30  →  4
40  →  1
